In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor
import os
!pip install xlsxwriter

# Load the Excel file and get sheet names
file_path = '/content/sps30_datalist.xlsx'
sheet_names = pd.ExcelFile(file_path).sheet_names

# Target sheets
target_sheets = ['ard', 'nacl', 'incense', 'oil']

# Filter the available sheets
valid_sheets = [sheet for sheet in sheet_names if sheet in target_sheets]

for sheet in valid_sheets:
    df = pd.read_excel(file_path, sheet_name=sheet)

    # Define targets and features
    y1 = df.iloc[:, [0]]
    y2 = df.iloc[:, [0]]
    y3 = df.iloc[:, [0]]
    y4 = df.iloc[:, [0]]
    y5 = df.iloc[:, [0]]
    x1 = df.iloc[:, [1]]
    x2 = df.iloc[:, [2]]
    x3 = df.iloc[:, [3]]
    x4 = df.iloc[:, [4]]
    x5 = df.iloc[:, [4]]

    # Train-test split
    x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, test_size=0.2, random_state=42)
    x2_train, x2_test, y2_train, y2_test = train_test_split(x2, y2, test_size=0.2, random_state=42)
    x3_train, x3_test, y3_train, y3_test = train_test_split(x3, y3, test_size=0.2, random_state=42)
    x4_train, x4_test, y4_train, y4_test = train_test_split(x4, y4, test_size=0.2, random_state=42)
    x5_train, x5_test, y5_train, y5_test = train_test_split(x5, y5, test_size=0.2, random_state=42)

    # Scaling
    scaler1 = StandardScaler()
    scaler2 = StandardScaler()
    scaler3 = StandardScaler()
    scaler4 = StandardScaler()
    scaler5 = StandardScaler()

    x1_train_scaled = scaler1.fit_transform(x1_train)
    x1_scaled = scaler1.transform(x1)

    x2_train_scaled = scaler2.fit_transform(x2_train)
    x2_scaled = scaler2.transform(x2)

    x3_train_scaled = scaler3.fit_transform(x3_train)
    x3_scaled = scaler3.transform(x3)

    x4_train_scaled = scaler4.fit_transform(x4_train)
    x4_scaled = scaler4.transform(x4)

    x5_train_scaled = scaler5.fit_transform(x5_train)
    x5_scaled = scaler5.transform(x5)

    # XGBoost hyperparameter tuning
    param_grid = {
        'n_estimators': [50, 100, 150],
        'learning_rate': [0.01, 0.1, 0.2],
        'max_depth': [3, 4, 5]
    }

    def train_best_model(x_train, y_train):
        xgb = XGBRegressor(random_state=42, objective='reg:squarederror')
        grid = GridSearchCV(xgb, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1)
        grid.fit(x_train, y_train)
        return grid.best_estimator_, grid.best_params_

    # Train and predict for each input
    model1, _ = train_best_model(x1_train_scaled, y1_train)
    model2, _ = train_best_model(x2_train_scaled, y2_train)
    model3, _ = train_best_model(x3_train_scaled, y3_train)
    model4, _ = train_best_model(x4_train_scaled, y4_train)
    model5, _ = train_best_model(x5_train_scaled, y4_train)

    y1_pred = model1.predict(x1_scaled)
    y2_pred = model2.predict(x2_scaled)
    y3_pred = model3.predict(x3_scaled)
    y4_pred = model4.predict(x4_scaled)
    y5_pred = model4.predict(x5_scaled)

    # Convert predictions to DataFrames
    y1_pred_df = pd.DataFrame(y1_pred, columns=y1_test.columns)
    y2_pred_df = pd.DataFrame(y2_pred, columns=y2_test.columns)
    y3_pred_df = pd.DataFrame(y3_pred, columns=y3_test.columns)
    y4_pred_df = pd.DataFrame(y4_pred, columns=y4_test.columns)
    y5_pred_df = pd.DataFrame(y5_pred, columns=y5_test.columns)

    # Metrics
    metrics = lambda y_true, y_pred: (
        mean_absolute_error(y_true, y_pred),
        mean_squared_error(y_true, y_pred),
        r2_score(y_true, y_pred)
    )

    mae1, mse1, r2_1 = metrics(y1, y1_pred)
    mae2, mse2, r2_2 = metrics(y2, y2_pred)
    mae3, mse3, r2_3 = metrics(y3, y3_pred)
    mae4, mse4, r2_4 = metrics(y4, y4_pred)
    mae5, mse5, r2_5 = metrics(y5, y5_pred)

    # Save to Excel
    output_file = f"{sheet}_XGBoost_Output.xlsx"
    with pd.ExcelWriter(output_file, engine='xlsxwriter') as writer:
        y1_pred_df.to_excel(writer, sheet_name="Predictions1", index=False)
        pd.DataFrame({"Metric": ["MAE", "MSE", "R2"], "Value": [mae1, mse1, r2_1]}).to_excel(writer, sheet_name="Metrics1", index=False)

        y2_pred_df.to_excel(writer, sheet_name="Predictions2", index=False)
        pd.DataFrame({"Metric": ["MAE", "MSE", "R2"], "Value": [mae2, mse2, r2_2]}).to_excel(writer, sheet_name="Metrics2", index=False)

        y3_pred_df.to_excel(writer, sheet_name="Predictions3", index=False)
        pd.DataFrame({"Metric": ["MAE", "MSE", "R2"], "Value": [mae3, mse3, r2_3]}).to_excel(writer, sheet_name="Metrics3", index=False)

        y4_pred_df.to_excel(writer, sheet_name="Predictions4", index=False)
        pd.DataFrame({"Metric": ["MAE", "MSE", "R2"], "Value": [mae4, mse4, r2_4]}).to_excel(writer, sheet_name="Metrics4", index=False)

        y5_pred_df.to_excel(writer, sheet_name="Predictions5", index=False)
        pd.DataFrame({"Metric": ["MAE", "MSE", "R2"], "Value": [mae5, mse5, r2_5]}).to_excel(writer, sheet_name="Metrics5", index=False)


    print(f"Completed for sheet: {sheet} -> Saved to {output_file}")

Completed for sheet: oil -> Saved to oil_XGBoost_Output.xlsx
Completed for sheet: ard -> Saved to ard_XGBoost_Output.xlsx
Completed for sheet: incense -> Saved to incense_XGBoost_Output.xlsx
Completed for sheet: nacl -> Saved to nacl_XGBoost_Output.xlsx
